# Dashboards Deep Dive: Advanced Features

Learn advanced dashboard features for professional data visualizations and reporting.

## What You'll Learn

✅ Query-based parameters and dynamic filters  
✅ Create reusable datasets  
✅ Manage permissions and sharing  
✅ Schedule and subscribe to dashboard updates  
✅ Integrate with Genie Spaces  

---

## Why Advanced Dashboards?

**Basic dashboards** show static visualizations.

**Advanced dashboards** provide:
- Dynamic filtering with parameters
- Reusable query datasets
- Scheduled refreshes and alerts
- Granular access control
- Natural language integration

---

## Table of Contents

1. [Query-Based Parameters](#parameters)
2. [Dashboard Filters](#filters)
3. [Reusable Datasets](#datasets)
4. [Permissions and Sharing](#sharing)
5. [Scheduling and Subscriptions](#scheduling)
6. [Genie Space Integration](#genie)

---

**References:**
- [Query-Based Parameters](https://docs.databricks.com/aws/en/dashboards/tutorials/query-based-params)
- [Dashboard Filters](https://docs.databricks.com/aws/en/dashboards/filters)
- [Datasets](https://docs.databricks.com/aws/en/dashboards/datasets)
- [Sharing](https://docs.databricks.com/aws/en/dashboards/share)
- [Scheduling](https://docs.databricks.com/aws/en/dashboards/schedule-subscribe)

## 1. Query-Based Parameters <a id="parameters"></a>

### What are Parameters?

Parameters make dashboards dynamic - users can filter and customize views without editing queries.

### Types of Parameters

**1. Static Parameters** - Fixed list of values
```sql
-- Define parameter: factory_filter
-- Type: Dropdown
-- Values: ["All", "TechHub Seattle", "Innovation Austin", "CloudForge Portland"]

SELECT * FROM sensor_enriched
WHERE factory_name = '{{factory_filter}}'
```

**2. Query-Based Parameters** - Dynamic values from a query
```sql
-- Parameter query to populate dropdown:
SELECT DISTINCT factory_name 
FROM sensor_enriched 
ORDER BY factory_name
```

**3. Date Range Parameters**
```sql
SELECT * FROM sensor_enriched
WHERE DATE(timestamp) BETWEEN '{{start_date}}' AND '{{end_date}}'
```

### Creating Parameters

**Step 1: Create Parameter Query**
```sql
-- Query: get_factories
SELECT DISTINCT factory_name, factory_name as label
FROM default.db_crash_course.sensor_enriched
ORDER BY factory_name
```

**Step 2: Use in Main Query**
```sql
-- Query: factory_metrics
SELECT 
  factory_name,
  COUNT(DISTINCT device_id) as device_count,
  AVG(temperature) as avg_temp,
  COUNT(*) as reading_count
FROM default.db_crash_course.sensor_enriched
WHERE factory_name = '{{factory_parameter}}'
  AND DATE(timestamp) >= '{{start_date}}'
GROUP BY factory_name
```

**Step 3: Link Parameter to Query**
- In query editor, add parameter widget
- Select "Query-based"
- Choose parameter query
- Set default value

### Parameter Best Practices

✅ **Use query-based parameters** for dynamic lists  
✅ **Add "All" option** when appropriate  
✅ **Set sensible defaults** (e.g., last 30 days)  
✅ **Label parameters clearly** for end users  
✅ **Test edge cases** (empty results, null values)


## 2. Dashboard Filters <a id="filters"></a>

### Filter Types

**1. Dropdown Filter**
```sql
-- Create filter for region
SELECT DISTINCT region FROM sensor_enriched ORDER BY region
```

**2. Multi-Select Filter**
```sql
-- Allow multiple factory selection
WHERE factory_name IN ({{factories}})
```

**3. Date Range Filter**
```sql
WHERE DATE(timestamp) BETWEEN {{start_date}} AND {{end_date}}
```

**4. Text Search Filter**
```sql
WHERE device_id LIKE '%{{search_text}}%'
```

### Filter Patterns

**Handling "All" Selection:**
```sql
SELECT * FROM sensor_enriched
WHERE (
  '{{factory_filter}}' = 'All' 
  OR factory_name = '{{factory_filter}}'
)
```

**Multi-Select with IN:**
```sql
WHERE factory_name IN ({{factory_list}})
-- Parameter type: Multi-select dropdown
```

---

## 3. Reusable Datasets <a id="datasets"></a>

### What are Datasets?

**Datasets** are reusable query results that can be shared across multiple dashboards.

### Benefits:
- **Performance**: Query once, use many times
- **Consistency**: Same data across dashboards
- **Maintenance**: Update query in one place
- **Access Control**: Manage permissions centrally

### Creating a Dataset

**Step 1: Create Source Query**
```sql
-- Query: iot_factory_metrics_dataset
SELECT 
  factory_name,
  region,
  DATE(timestamp) as reading_date,
  COUNT(DISTINCT device_id) as device_count,
  AVG(temperature) as avg_temperature,
  MAX(temperature) as max_temperature,
  SUM(CASE WHEN temperature > 85 THEN 1 ELSE 0 END) as critical_readings
FROM default.db_crash_course.sensor_enriched
WHERE DATE(timestamp) >= CURRENT_DATE - 30
GROUP BY factory_name, region, DATE(timestamp)
```

**Step 2: Publish as Dataset**
- Save query
- Click "Publish as Dataset"
- Set refresh schedule
- Configure permissions

**Step 3: Use in Dashboards**
```sql
-- Reference the dataset instead of base tables
SELECT * FROM {{dataset:iot_factory_metrics_dataset}}
WHERE region = '{{region_filter}}'
```

### Dataset Best Practices

✅ **Pre-aggregate** for performance  
✅ **Schedule refreshes** appropriately  
✅ **Document** dataset purpose and structure  
✅ **Version** dataset definitions  
✅ **Monitor** query performance  

---

## 4. Permissions and Sharing <a id="sharing"></a>

### Permission Levels

**1. View** - Can view dashboard only  
**2. Run** - Can view and refresh  
**3. Edit** - Can modify dashboard  
**4. Manage** - Full control including permissions  

### Sharing Workflows

**Share with Users:**
1. Open dashboard
2. Click "Share"
3. Add users/groups
4. Set permission level
5. Optionally send notification

**Share with External Users:**
- Publish dashboard with token
- Embed in external applications
- Set expiration dates

**Share via Email:**
- Schedule dashboard snapshots
- Email to distribution list
- Include filters in snapshot

### Access Control Best Practices

✅ **Use groups** not individual users  
✅ **Least privilege** - grant minimum needed  
✅ **Review regularly** - audit access quarterly  
✅ **Document** who needs what and why  

---

## 5. Scheduling and Subscriptions <a id="scheduling"></a>

### Refresh Schedules

**Set Automatic Refresh:**
1. Open dashboard
2. Click Schedule
3. Set frequency (hourly, daily, weekly)
4. Choose time and timezone
5. Enable/disable as needed

**Example Schedules:**
- **Real-time**: Every 5 minutes
- **Operational**: Hourly during business hours
- **Executive**: Daily at 6 AM
- **Weekly reports**: Monday mornings

### Email Subscriptions

**Subscribe Users:**
```
Schedule: Daily at 8 AM PST
Recipients: operations-team@company.com
Include: PDF snapshot
Filters: Last 24 hours, All factories
```

**Conditional Alerts:**
```sql
-- Only send if anomalies detected
SELECT 
  COUNT(*) as anomaly_count,
  AVG(temperature) as avg_temp
FROM sensor_enriched
WHERE temperature > 85
  AND DATE(timestamp) = CURRENT_DATE
HAVING COUNT(*) > 100
```

### Slack Integration

- Connect Slack workspace
- Choose channel
- Set notification triggers
- Customize message format

---

## 6. Genie Space Integration <a id="genie"></a>

### Connect Dashboard to Genie

Genie can answer natural language questions about your dashboard data.

**Setup Steps:**
1. Create Genie Space
2. Add dashboard datasets as trusted assets
3. Configure business context
4. Enable for users

**Example Queries:**
- "What's the average temperature across all factories?"
- "Show me factories with critical readings today"
- "Compare device counts between regions"

**Benefits:**
- Ad-hoc analysis without SQL
- Exploratory data analysis
- Business user self-service

---

## Summary

In this notebook, you learned:

✅ **Query-based parameters** - Dynamic dropdowns and filters  
✅ **Dashboard filters** - Multi-select, date range, text search  
✅ **Reusable datasets** - Share queries across dashboards  
✅ **Permissions** - Control access with granular permissions  
✅ **Scheduling** - Automatic refreshes and email subscriptions  
✅ **Genie integration** - Natural language dashboard queries  

### Key Takeaways:

1. **Parameters make dashboards interactive** - Users customize views
2. **Datasets improve performance** - Query once, use many times
3. **Schedule refreshes appropriately** - Balance freshness and cost
4. **Use groups for permissions** - Easier to manage
5. **Integrate with Genie** - Enable natural language exploration

### Next Steps:

- Build an interactive dashboard with parameters
- Create a reusable dataset
- Set up scheduled email reports
- Connect dashboard to Genie Space

---

**Additional Resources:**
- [Query Parameters Tutorial](https://docs.databricks.com/aws/en/dashboards/tutorials/query-based-params)
- [Dashboard Filters](https://docs.databricks.com/aws/en/dashboards/filters)
- [Datasets Documentation](https://docs.databricks.com/aws/en/dashboards/datasets)
- [Sharing Dashboards](https://docs.databricks.com/aws/en/dashboards/share)
